In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# own Modules 
from models import LstmMse
from data_loader import CrossValidationProvider, DataScaler, DataSet
from trainer import Trainer
from loss_module import LossMse

Using TensorFlow backend.


## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change step_size in cycling_lr
- Change filed_location

## Hyperparameters

In [40]:
param = {
    "data" : {
        "path" : '../../../data/phm_data_challenge/01_M01_DC_prediction_1.csv' ,
    },
    "preprocessing" : {
        "droped_features": ["ID", "stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 13,
        "n_hidden_lstm" : [50, 100, 200],
        "sequence_size" : [50, 100, 200],
        "batch_size" : 8,
        "lstm_layer" : [2, 4],
        "n_hidden_fc": [25, 50, 100],
        "dropout_rate": 0.2
        
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # step_size is the number of training iterations (total samples/batch_size) per half cycle. 
        # Authors suggest setting step_size 2-8 x training iterations in epoch.
        "step_size" : (12500/8)*2, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.016, 
        "max_lr" :0.75
    },
    "training": {
        "stake_training_data": 0.75,
        "total_number" : 30000,
        "n_folds_cv": 7,
        "n_epochs" : 100,
        "patience" : 5,
    },
    "filed_location": {
        "trained_model" : "../../../models/MSE_model/phm_data",
        "history" : "../../visualisation/files/history_training/phm_data.csv"
    }
}

In [43]:
class CrossValidationProvider():
    def __init__(self, path, no_folds, amount_data, stake_training_data, ignored_features):
        self.path = path
        self.train_data = None
        self.test_data = None
        self.no_folds = no_folds
        self.amount_data = amount_data
        self.stake_training_data = stake_training_data
        self.ignored_features = ignored_features
        
    def split_training_test(self):
        # Read hole dataset und drop features
        dataset = pd.read_csv(self.path)
        dataset = dataset.drop(labels=self.ignored_features, axis=1)
        
        # Select training and test dataset
        sub_dataset = dataset.iloc[:self.amount_data,:]
        amount_training_data = round(len(sub_dataset)*self.stake_training_data)
        self.train_data = sub_dataset.iloc[:amount_training_data,:]
        self.test_data = sub_dataset.iloc[amount_training_data:,:]
        
    def split_dataset_in_folds(self):
        amount_each_fold = round(len(self.train_data) / self.no_folds)
        
        folds = []
        for fold_number in range(1,self.no_folds+1):
            fold = self.train_data.iloc[(fold_number-1)*amount_each_fold:fold_number*amount_each_fold,:]
            folds.append(fold)
        return folds
        
    def provide_data(self):
        self.split_training_test()
        folds = self.split_dataset_in_folds()
        return self.test_data, folds 

In [ ]:
class DataScaler():
    def __init__(self, features_not_to_scale):
        self.scaler = StandardScaler()
        self.features_not_to_scale = features_not_to_scale
    
    def scale_data(self, train_data, validation_data):
        if len(self.features_not_to_scale) == 0:
            # Initialise standard scaler
            self.scaler.fit(train_data)
            # Transform data
            train_scaled = self.scaler.transform(train_data)
            validation_scaled = self.scaler.transform(validation_data)
        
        else:
            # seperate categorical and continous features
            categorical_features_train = train_data.loc[:, self.features_not_to_scale]
            continous_features_train = train_data.drop(labels=self.features_not_to_scale, axis=1)
            categorical_features_validation = validation_data.loc[:, self.features_not_to_scale]
            continous_features_validation = validation_data.drop(labels=self.features_not_to_scale, axis=1)

            # Initialise standard scaler
            self.scaler.fit(continous_features_train)
            # Transform data
            continous_train_scaled = self.scaler.transform(continous_features_train)
            continous_validation_scaled = self.scaler.transform(continous_features_validation)

            # Combine categorical and scaled features 
            train_scaled = np.concatenate((continous_train_scaled, categorical_features_train), axis=1)
            validation_scaled = np.concatenate((continous_validation_scaled, categorical_features_validation), axis=1)
        return train_scaled, validation_scaled

## Split Data into folds

In [44]:
cv_provider = CrossValidationProvider(path=param["data"]["path"], 
                                      no_folds=param["training"]["n_folds_cv"], 
                                      amount_data=param["training"]["total_number"],
                                      stake_training_data = param["training"]["stake_training_data"],
                                      ignored_features = param['preprocessing']['droped_features']
                                     )
test_data, folds = cv_provider.provide_data()

## Cross Validation Training

In [62]:
loss_iterations = []
for iteration in range (1, param["training"]["n_folds_cv"]):
    # Select folds for current iteration
    training_folds = folds[:iteration]
    validation_fold = folds[iteration:iteration+1]
    print("Training Data : Fold 1-" + str(iteration))
    print("Validation Data : Fold "+ str(iteration+1))
    
    # Concate data of training folds and unpack validation data
    raw_training_data
    raw_validation_data = validation_fold[0]
    
    # Scale training data and validation data (validation data with mean and variance of training data)
    scaler = DataScaler(features_not_to_scale= param['preprocessing']['features_not_to_scale']
                       )
    train_data, validation_data = scaler.scale_data()
    
    # 
    for n_lstm_layer in param["model"]["lstm_layer"]:
        for sequence_size  in param["model"]["sequence_size"]:
            # Initialize DataSet
            dataset_train = DataSet(train_data, timesteps=param["model"]["sequence_size"])
            dataset_validation = DataSet(validation_data, timesteps=param["model"]["sequence_size"])

            # Initialize DataLoader
            data_loader_training = DataLoader(dataset_train, 
                                              batch_size=param["model"]["batch_size"], 
                                              num_workers=0, 
                                              shuffle=True, 
                                              drop_last=True
                                             )
            data_loader_validation = DataLoader(dataset_validation, 
                                                batch_size=param["model"]["batch_size"], 
                                                num_workers=0, 
                                                shuffle=True, 
                                                drop_last=True
                                               )
            for n_hidden_lstm in param["model"]["n_hidden_lstm"]:
                for n_hidden_fc in param["model"]["n_hidden_fc"]:
                    
                    print("Start with new hyperparameters in grid search: ")
                    print("Sequence_size: {}".format(sequence_size))
                    print("Number LSTM Layers: {}".format(n_lstm_layer))
                    print("LSTM Number Hidden Dimensions: {}".format(n_hidden_lstm))
                    print("FC NN Number Hidden Dimensions: {}".format(n_hidden_fc))

                    # Create lists to save training loss and validation loss of each epoch
                    hist_loss = []
                    torch.manual_seed(0)
                    model = LstmMse(batch_size=param['model']['batch_size'], 
                                    input_dim=param['model']['input_size'], 
                                    n_hidden_lstm=n_hidden_lstm, 
                                    n_layers=n_lstm_layer,
                                    dropout_rate= param['model']['dropout_rate'],
                                    n_hidden_fc=n_hidden_fc
                                    )

    
                    # Define Loss Function
                    criterion = LossMse(param["model"]["input_size"], param["model"]["batch_size"])

                    # Initialize Optimizer and Cyclic Learning Rate Scheduler
                    optimizer = torch.optim.SGD(model.parameters(), lr=1.)  
                    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                                                  base_lr=param['cycling_lr']['base_lr'], 
                                                                  max_lr=param['cycling_lr']['max_lr'], 
                                                                  step_size_up=param['cycling_lr']['step_size'], 
                                                                  mode=param['cycling_lr']['mode'],
                                                                  gamma=param['cycling_lr']['gamma']
                                                                  )
                    trainer = Trainer(model=model,
                                      optimizer=optimizer,
                                      scheduler=scheduler,
                                      scheduler_active = param["cycling_lr"]["scheduler_active"],
                                      criterion=criterion, 
                                      location_model=param["filed_location"]["trained_model"], 
                                      location_stats=param["filed_location"]["history"], 
                                      patience=param['training']['patience']
                                     )
                    
                    # Measure training time for current configuration
                    start = time.time()

                    for epoch in range(param['training']['n_epochs']):
                        # Train
                        mean_epoch_training_loss = trainer.train(data_loader_training)

                        # Evaluate
                        mean_epoch_validation_loss = trainer.evaluate(data_loader_validation, hist_loss, epoch)

                        # Cache History
                        trainer.cache_history_training(hist_loss, epoch, mean_epoch_training_loss, mean_epoch_validation_loss)

                        # Save model if its the best one since the last change in configuration of hyperparameters
                        status_ok = trainer.save_model(epoch, mean_epoch_validation_loss, param['model']['input_size'], 
                                                       n_lstm_layer, n_hidden_lstm, n_hidden_fc, sequence_size)
                        if not status_ok:
                            #ToDo: Safe loss and configuration to list
                            break

                    # Time in minutes
                    execution_time = (time.time() - start)/60

                    # Save training statistics 
                    trainer.save_statistic(hist_loss, sequence_size, n_lstm_layer, n_hidden_lstm, n_hidden_fc, execution_time)         
    

Training Data : Fold 1-1
Validation Data : Fold 2
Training Data : Fold 1-2
Validation Data : Fold 3
Training Data : Fold 1-3
Validation Data : Fold 4
Training Data : Fold 1-4
Validation Data : Fold 5
Training Data : Fold 1-5
Validation Data : Fold 6
Training Data : Fold 1-6
Validation Data : Fold 7


## Training Loop
An epoch consists of a learning cycle over all batches of training data and an evaluation of the most recent model with the testing data. 

In [54]:
# Create lists to save training loss and validation loss of each epoch
hist_loss = []
torch.manual_seed(0)
print("Start training.")

for epoch in range(param['training']['n_epochs']):
    # Train with batches 
    mean_epoch_training_loss = trainer.train(data_loader_training)

    # Evaluate
    mean_epoch_validation_loss = trainer.evaluate(data_loader_validation, hist_loss, epoch)

    # Cache History
    trainer.cache_history_training(hist_loss, epoch, mean_epoch_training_loss, mean_epoch_validation_loss)

    # Save model if its the best one since the last change in configuration of hyperparameters
    status_ok = trainer.save_model(epoch, mean_epoch_validation_loss, param['model']['input_size'], 
                                   param['model']['lstm_layer'], param['model']['n_hidden_lstm'], 
                                   param['model']['n_hidden_fc'], param["model"]["sequence_size"])
    if not status_ok:
        break

# Safe results to csv file
df = pd.DataFrame(hist_loss)
df.to_csv(param["filed_location"]["history"], sep=";", index=False)

/Users/carlovoss/jupyter_notebooks/masterarbeit/venv_pm/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor(0.1124, grad_fn=<DivBackward0>)
tensor(0.0689, grad_fn=<DivBackward0>)
tensor(0.0072, grad_fn=<DivBackward0>)
tensor(0.0618, grad_fn=<DivBackward0>)
tensor(0.0099, grad_fn=<DivBackward0>)
tensor(0.0098, grad_fn=<DivBackward0>)
tensor(0.0319, grad_fn=<DivBackward0>)
tensor(0.0424, grad_fn=<DivBackward0>)
tensor(0.0145, grad_fn=<DivBackward0>)
tensor(0.0099, grad_fn=<DivBackward0>)
tensor(0.0107, grad_fn=<DivBackward0>)
tensor(0.0708, grad_fn=<DivBackward0>)
tensor(0.0123, grad_fn=<DivBackward0>)
tensor(0.0102, grad_fn=<DivBackward0>)
tensor(0.0082, grad_fn=<DivBackward0>)
tensor(0.0088, grad_fn=<DivBackward0>)
tensor(0.0757, grad_fn=<DivBackward0>)
tensor(0.0753, grad_fn=<DivBackward0>)
tensor(0.0665, grad_fn=<DivBackward0>)
tensor(0.0100, grad_fn=<DivBackward0>)
tensor(0.0084, grad_fn=<DivBackward0>)
tensor(0.0147, grad_fn=<DivBackward0>)
tensor(0.0098, grad_fn=<DivBackward0>)
tensor(0.0152, grad_fn=<DivBackward0>)
tensor(0.0513, grad_fn=<DivBackward0>)
tensor(0.0761, grad_fn=<D

Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 40

KeyboardInterrupt: 

## Visualization of cyclic learning rate

In [ ]:
x = range(len(lr_find_lr))
data = pd.DataFrame(data={'y': lr_find_lr, 'x': x})
f, ax = plt.subplots(figsize=(7, 7))
sns.lineplot(x=data.x, y=data.y, ax=ax)
plt.show()